In [1]:
import os
import numpy as np
import pandas as pd

RAW_PATH = "/data2/zhoujb/project/cowpea_project/rawData/"
SNP_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/snpDir/"
CLUSTER_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/SNPMMSeqCluster/"
RES_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/Result/"
TEMP_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/templateSNP/"

In [2]:
output_file = open(os.path.join(RES_PATH, "Haplotype_Basepair_Info.tsv"), "w")
print("\t".join(["Haplotype_ID", "SNP_Pos", "Haplotype"]), file=output_file)

file_list = [x for x in os.listdir(CLUSTER_PATH) if x.endswith(".final")]
for file_name in file_list:
    item_list = file_name.split("_")
    Chr = item_list[0]
    Start = item_list[1]
    End = item_list[2]
    region_index = "{}_{}_{}".format(Chr, Start, End)\
    
    raw_snp_txt =  pd.read_table(os.path.join(SNP_PATH, "{}.txt".format(region_index)))

    mmseq_res_path = os.path.join(CLUSTER_PATH, file_name)
    cluster_df = pd.read_table(mmseq_res_path)
    for group in cluster_df.groupby("Class_Rep"):
        group_name, group_data = group
        group_sample_list = group_data["Sample_name"].to_list()
        
        num = group_data.shape[0]
        total = cluster_df.shape[0]
        percentage = num / total
        if percentage <= 0.25:
            geno_pref = "NH"
        elif percentage <= 0.5:
            geno_pref = "VH"
        elif percentage <= 0.75:
            geno_pref = "IH"
        else:
            geno_pref = "PH"
        
        hap_id = "{}_{}_{}_{}".format(region_index, num, geno_pref, group_name)

        snp_pos_list = [str(x) for x in raw_snp_txt["Pos"].to_list()]
        
        sample_sel_df = raw_snp_txt[group_sample_list].copy()
        haplotype_list = []
        for index in sample_sel_df.index:
            most_freq_nn = sample_sel_df.loc[index].value_counts().idxmax()
            haplotype_list.append(most_freq_nn)

        print("\t".join([hap_id, ",".join(snp_pos_list), ",".join(haplotype_list)]), file=output_file)
        del hap_id, snp_pos_list, haplotype_list

output_file.close()
print("ALL DONE")

ALL DONE


In [2]:
with open(os.path.join(RES_PATH, "Haplotype_Basepair_Info.tsv")) as f:
    f.readline()
    for line in f:
        line = line.rstrip().split("\t")
        hap_id = line[0]
        cHTB_index = "_".join(hap_id.split("_")[:3])
        pos_list = line[1].split(",")
        val_list = line[2].split(",")
        if len(pos_list) != len(val_list):
            raise ValueError("{} {}".format(cHTB_index, hap_id))

        hap_id_dir = os.path.join(TEMP_PATH, "{}/".format(cHTB_index))
        if not os.path.exists(hap_id_dir):
            os.mkdir(hap_id_dir)

        output_file = open(os.path.join(hap_id_dir, "{}".format(hap_id)), "w")
        print("\t".join(["Index", "{}".format(hap_id)]), file=output_file)
        for x, y in zip(pos_list, val_list):
            print("\t".join([x, y]), file=output_file)

        output_file.close()

print("ALL DONE")  

ALL DONE
